# 911 Calls Data Visualization

911 call data from [Kaggle](https://www.kaggle.com/mchirico/montcoalert). The data contains the following fields:

* lat : String variable, Latitude
* lng: String variable, Longitude
* desc: String variable, Description of the Emergency Call
* zip: String variable, Zipcode
* title: String variable, Title
* timeStamp: String variable, YYYY-MM-DD HH:MM:SS
* twp: String variable, Township
* addr: String variable, Address
* e: String variable, Dummy variable (always 1)


** Import libraries and set %matplotlib inline.  (Chart display within Jupyter Notebook)**
 * ** numpy  **
 * ** pandas  **
 * ** visualization  **
 * ** plotly   ** (offine use for notebook)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)

** Load Dataset using panda's read_csv function called callsDataFrame **

In [ ]:
callsDataFrame = pd.read_csv('../input/911.csv')

** Check the info() of the callsDataFrame **

In [ ]:
callsDataFrame.info()

** Check the head of callsDataFrame **

In [ ]:
callsDataFrame.head()

** Lets find top 5 zipcodes for 911 calls. **

In [ ]:
callsDataFrame['zip'].value_counts().head(5)

** Lets find top 5 townships (twp) for 911 calls **

In [ ]:
callsDataFrame['twp'].value_counts().head(5)

** Take a look at the 'title' column, how many unique title codes are there? **

In [ ]:
# len(df['title'].unique())
callsDataFrame['title'].nunique()

## Creating new features

** In the titles column there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.** 

**For example, if the title column value is EMS: BACK PAINS/INJURY , the Reason column value would be EMS. **

In [ ]:
x = callsDataFrame['title'].iloc[0]

In [ ]:
x.split(':')[0]

In [ ]:
callsDataFrame['Reason'] = callsDataFrame['title'].apply(lambda title: title.split(':')[0])

In [ ]:
callsDataFrame['Reason'].head()

** What is the most common Reason for a 911 call based off of this new column? **

In [ ]:
callsDataFrame['Reason'].value_counts()

** Let's use seaborn to create a countplot of 911 calls by Reason. **

In [ ]:
sns.countplot(x='Reason', data=callsDataFrame)

** Let's use plotly to create bar chart of 911 calls by Reason. **

In [ ]:
data = [go.Bar(
            x=['EMS', 'Fire', 'Traffic'],
            y=callsDataFrame['Reason'].value_counts()
    )]

iplot(data, filename='basic-bar')

___
** Now let us begin to focus on time information. What is the data type of the objects in the timeStamp column? **

In [ ]:
type(callsDataFrame['timeStamp'].iloc[0])

** You should have seen that these timestamps are still strings. Use [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) to convert the column from strings to DateTime objects. **

In [ ]:
callsDataFrame['timeStamp'] = pd.to_datetime(callsDataFrame['timeStamp'])

In [ ]:
callsDataFrame['timeStamp'].iloc[0]

** You can now grab specific attributes from a Datetime object by calling them. For example:**

    time = df['timeStamp'].iloc[0]
    time.hour

**You can use Jupyter's tab method to explore the various attributes you can call. Now that the timestamp column are actually DateTime objects, use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column, reference the solutions if you get stuck on this step.**

In [ ]:
time = callsDataFrame['timeStamp'].iloc[0]
time.hour

In [ ]:
time.month

In [ ]:
time.dayofweek

In [ ]:
callsDataFrame['Hour'] = callsDataFrame['timeStamp'].apply(lambda time: time.hour)

In [ ]:
callsDataFrame['Month'] = callsDataFrame['timeStamp'].apply(lambda time: time.month)

In [ ]:
callsDataFrame['Day Of Week'] = callsDataFrame['timeStamp'].apply(lambda time: time.dayofweek)

In [ ]:
callsDataFrame.head(5)

** Notice how the Day of Week is an integer 0-6. Use the .map() with this dictionary to map the actual string names to the day of the week: **

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
callsDataFrame['Day Of Week'] = callsDataFrame['Day Of Week'].map(dmap)

In [ ]:
callsDataFrame.head()

** Now use seaborn to create a countplot of the Day of Week column with the hue based off of the Reason column. **

In [ ]:
sns.countplot(x = 'Day Of Week', data = callsDataFrame, hue = 'Reason', palette="rocket")

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

**Now do the same for Month:**

In [ ]:
sns.countplot(x = 'Month', data = callsDataFrame, hue = 'Reason', palette="rocket")

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

** Here noticed that, it was missing some Months, let's see if we can fill in the missing months**

** Now create a gropuby object called byMonth, where you group the DataFrame by the month column and use the count() method for aggregation.**

In [ ]:
byMonth = callsDataFrame.groupby('Month').count()

In [ ]:
byMonth.head()

** Now create a simple plot off of the dataframe indicating the count of calls per month. **

In [ ]:
byMonth['lat'].plot()

In [ ]:
sns.countplot(x='Month', data=callsDataFrame, palette='rocket')

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

** Now see if you can use seaborn's lmplot() to create a linear fit on the number of calls per month. Keep in mind you may need to reset the index to a column. **

In [ ]:
## Resert Or convert the Month index to Column
byMonth.reset_index()

In [ ]:
## Here I'm using seaborn liner plot passing Month Column as DATA
sns.lmplot(x='Month', y='twp', data=byMonth.reset_index())

**Create a new column called 'Date' that contains the date from the timeStamp column. You'll need to use apply along with the .date() method. ** 

In [ ]:
callsDataFrame['timeStamp'].iloc[0]

In [ ]:
callsDataFrame['timeStamp'].iloc[0].date()

In [ ]:
# Creating new 'Date' column using timeStamp column
callsDataFrame['Date'] = callsDataFrame['timeStamp'].apply(lambda ts : ts.date())

In [ ]:
callsDataFrame['Date'].head()

In [ ]:
## Here, we can see Date Column inside DataFrame
callsDataFrame.head()

** Now groupby this Date column with the count() aggregate and create a plot of counts of 911 calls.**

In [ ]:
callsDataFrame.groupby('Date').count()['lat'].plot()
plt.tight_layout()

** Now recreate this plot but create 3 separate plots with each plot representing a Reason for the 911 call**

In [ ]:
callsDataFrame[callsDataFrame['Reason'] == 'Traffic'].groupby('Date').count()['lat'].plot()
plt.title('Traffic')
plt.tight_layout()

In [ ]:
callsDataFrame[callsDataFrame['Reason'] == 'Fire'].groupby('Date').count()['lat'].plot()
plt.title('Fire')
plt.tight_layout()

In [ ]:
callsDataFrame[callsDataFrame['Reason'] == 'EMS'].groupby('Date').count()['lat'].plot()
plt.title('EMS')
plt.tight_layout()

____
** Now let's move on to creating  heatmaps with seaborn and our data. We'll first need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. There are lots of ways to do this, but I would recommend trying to combine groupby with an [unstack](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method. Reference the solutions if you get stuck on this!**

In [ ]:
# Multilevel index count
callsDataFrame.groupby(by=['Day Of Week', 'Hour']).count().head()

In [ ]:
callsDataFrame.groupby(by=['Day Of Week', 'Hour']).count()['Reason'].head()

In [ ]:
# Having matric level table (* There is an alternate ways like pivot table)
callsDataFrame.groupby(by=['Day Of Week', 'Hour']).count()['Reason'].unstack()

In [ ]:
dayHour = callsDataFrame.groupby(by=['Day Of Week', 'Hour']).count()['Reason'].unstack()

** Now create a HeatMap using this new DataFrame. **

In [ ]:
## Day Of Week Vs Hour
sns.heatmap(dayHour)

** Now create a clustermap using this DataFrame. **

In [ ]:
sns.clustermap(dayHour)

** Now repeat these same plots and operations, for a DataFrame that shows the Month as the column. **

In [ ]:
dayMonth = callsDataFrame.groupby(by=['Day Of Week', 'Month']).count()['Reason'].unstack()

In [ ]:
sns.heatmap(dayMonth)

In [ ]:
sns.clustermap(dayMonth)

**That's It!**